In [12]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/traffic-sign-classification/'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/traffic-sign-classification


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [6]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%%d-%H%M%S'))
  tensorboard_callback=tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=tensorboard_callback
      )
  
  return model


def predict(model_trained, X_test, y_test, scoring=accuracy_score ):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm,y_pred)

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation = 'relu'),
  Dropout(0.3),

  Dense(1024, activation = 'relu'),
  Dropout(0.3),

  Dense(num_classes, activation = 'softmax'),
])


In [9]:
model = get_cnn_v5(input_shape,num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 20ms/step - loss: 2.6061 - accuracy: 0.2887 - val_loss: 0.9613 - val_accuracy: 0.7006
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.8002 - accuracy: 0.7463 - val_loss: 0.2091 - val_accuracy: 0.9376
Epoch 3/5
272/272 [==============================] - 5s 18ms/step - loss: 0.3672 - accuracy: 0.8857 - val_loss: 0.0894 - val_accuracy: 0.9722
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2308 - accuracy: 0.9306 - val_loss: 0.0663 - val_accuracy: 0.9801
Epoch 5/5
272/272 [==============================] - 5s 18ms/step - loss: 0.1898 - accuracy: 0.9436 - val_loss: 0.0337 - val_accuracy: 0.9902


0.9689342403628118

In [10]:
model_trained.evaluate(X_test,y_test)

138/138 [==============================] - 0s 4ms/step - loss: 0.2565 - accuracy: 0.9689


[0.25647637248039246, 0.9689342379570007]

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024, activation = 'relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024, activation = 'relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation = 'softmax'),
])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
      )
  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [29]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 110.0, 'dropout_cnn_block_one': 0.48740500249100893, 'dropout_cnn_block_three': 0.4987601775105097, 'dropout_cnn_block_two': 0.34043297134859746, 'dropout_dense_block_one': 0.3579064363548787, 'dropout_dense_block_two': 0.41339556878215244}
accuracy=0.9356009364128113
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3831404787350065, 'dropout_cnn_block_three': 0.37802131663266975, 'dropout_cnn_block_two': 0.4206370239589887, 'dropout_dense_block_one': 0.33592297986060066, 'dropout_dense_block_two': 0.38317584763552637}
accuracy=0.962358295917511
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.4653326956523011, 'dropout_cnn_block_three': 0.3008309769657516, 'dropout_cnn_block_two': 0.37186278298424247, 'dropout_dense_block_one': 0.5218874817174161, 'dropout_dense_block_two': 0.42713972840208053}
accuracy=0.9580498933792114
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.380393292444319, 'dropout_cnn_block_three': 0.3884860177719569, 'dropout_cnn_block_two': 0.4802453989